In [1]:
import pint
import xarray as xr
from dask.distributed import Client
from seapopym.configuration.no_transport.parameter import ForcingParameters
from seapopym.configuration.parameters.parameter_forcing import ForcingUnit
from seapopym.standard.units import StandardUnitsLabels

from seapopym_optimization import (
    FunctionalGroupOptimizeNoTransport,
    GeneticAlgorithm,
    GeneticAlgorithmParameters,
    NoTransportCostFunction,
    Observation,
    Parameter,
    constraint,
)

Load forcing.


In [2]:
time_start, time_end = "1998-01-01", "2022-01-01"

bats_data = xr.open_dataset("../notebooks/1_data_processing/1_1_Forcing/bats_cmems.zarr", engine="zarr")
bats_data["T"].attrs["units"] = StandardUnitsLabels.temperature.units
bats_data = bats_data.sel(time=slice(time_start, time_end))
_ = bats_data.load()

cafe_npp = xr.open_dataset("../notebooks/1_data_processing/1_1_Forcing/bats_cafe.zarr", engine="zarr")
cafe_npp = cafe_npp.sel(time=slice(time_start, time_end))
cafe_npp = cafe_npp.dropna("time")
cafe_npp = cafe_npp.resample(time="D").interpolate("linear")
cafe_npp.latitude.attrs = {"units": "degrees_north", "long_name": "latitude", "axis": "Y"}
cafe_npp.longitude.attrs = {"units": "degrees_east", "long_name": "longitude", "axis": "X"}
# cafe_npp = cafe_npp.rename({"lat": "latitude", "lon": "longitude"})
_ = cafe_npp.load()

In [3]:
bats_data

<xarray.Dataset> Size: 807kB
Dimensions:              (time: 8767, depth: 3, latitude: 1, longitude: 1)
Coordinates:
  * depth                (depth) float32 12B 1.0 2.0 3.0
  * latitude             (latitude) float64 8B 31.6
  * longitude            (longitude) float64 8B -64.2
  * time                 (time) datetime64[ns] 70kB 1998-01-01 ... 2022-01-01
Data variables: (12/13)
    T                    (time, depth, latitude, longitude) float32 105kB 20....
    U                    (time, depth, latitude, longitude) float32 105kB 0.1...
    V                    (time, depth, latitude, longitude) float32 105kB -0....
    mnkc_epi             (time, latitude, longitude) float32 35kB 0.7599 ... ...
    mnkc_hmlmeso         (time, latitude, longitude) float32 35kB 0.6979 ... ...
    mnkc_lmeso           (time, latitude, longitude) float32 35kB 1.093 ... 1...
    ...                   ...
    mnkc_mumeso          (time, latitude, longitude) float32 35kB 0.6176 ... ...
    mnkc_umeso           (time, latitude, longitude) float32 35kB 0.3953 ... ...
    npp                  (time, latitude, longitude) float32 35kB 446.1 ... 2...
    pelagic_layer_depth  (time, depth, latitude, longitude) float32 105kB 100...
    zeu                  (time, latitude, longitude) float32 35kB 66.99 ... 7...
    zooc                 (time, latitude, longitude) float32 35kB 0.3641 ... ...
Attributes:
    Conventions:       CF-1.11
    history:           Created on 2022-09-28
    institution:       CLS
    references:        http://www.cls.fr; http://www.seapodym.eu
    source:            SEAPODYM-LMTL 3.0.0
    subset:datasetId:  cmems_mod_glo_bgc_my_0.083deg-lmtl_PT1D-i_202211
    subset:date:       2024-06-12T03:45:47.386Z
    subset:productId:  GLOBAL_MULTIYEAR_BGC_001_033
    subset:source:     ARCO data downloaded from the Marine Data Store using ...
    title:             Global ocean low and mid trophic levels biomass conten...

Load observations.

First I multiply the observations by the average epipelagic layer depth (150m) to have a biomass in m2 rather than m3.


In [4]:
def update_layer(data: xr.DataArray, epipelagic_size):
    data = data.pint.quantify() * epipelagic_size
    layer_attrs = data["layer"].attrs
    data = data.assign_coords({"layer": [1]})
    data["layer"].attrs = layer_attrs
    return data


epipelagic_size = 150 * pint.application_registry("meter")

obs_bats = xr.open_dataset("../../seapopym-data/data/zooplankton_product/Bats_zooplankton.nc")
obs_bats = update_layer(obs_bats, epipelagic_size)
obs_bats = Observation(obs_bats)
observations = [obs_bats]

Create structure for SeapoPym simulation.


In [5]:
forcing_parameters = ForcingParameters(
    temperature=ForcingUnit(forcing=bats_data["T"], resolution=1 / 12),
    primary_production=ForcingUnit(forcing=cafe_npp["CAFE"], resolution=1 / 12),
)

2024-11-06 11:56:43,207 :: Seapodym ::  WARNING ::
|	CAFE unit is milligram / day / meter ** 2, it will be converted to kilogram / day / meter ** 2.



Setup the cost function.


In [6]:
functional_groups = [
    FunctionalGroupOptimizeNoTransport(
        name="D1N1",
        day_layer=1,
        night_layer=1,
        tr_rate=Parameter("D1N1_tr_rate", -1, 0),
        tr_max=Parameter("D1N1_tr_max", 0, 50),
        inv_lambda_rate=Parameter("D1N1_inv_lambda_rate", 0, 1),
        inv_lambda_max=Parameter("D1N1_inv_lambda_max", 0, 100),
        energy_coefficient=Parameter("D1N1_energy_coefficient", 0.05, 0.8),
    ),
    FunctionalGroupOptimizeNoTransport(
        name="D2N1",
        day_layer=2,
        night_layer=1,
        tr_rate=Parameter("D2N1_tr_rate", -1, 0),
        tr_max=Parameter("D2N1_tr_max", 0, 50),
        inv_lambda_rate=Parameter("D2N1_inv_lambda_rate", 0, 1),
        inv_lambda_max=Parameter("D2N1_inv_lambda_max", 0, 100),
        energy_coefficient=Parameter("D2N1_energy_coefficient", 0.05, 0.8),
    ),
]

In [7]:
cost_function = NoTransportCostFunction(
    functional_groups=functional_groups,
    forcing_parameters=forcing_parameters,
    observations=observations,
)

Set the genetic algorithm meta parameters.


In [8]:
genetic_algo_parameters = GeneticAlgorithmParameters(
    ETA=1,
    INDPB=0.1,
    CXPB=0.5,
    MUTPB=0.2,
    NGEN=20,
    POP_SIZE=20,
    cost_function_weight=(-1,),
)

Add a constraint to limit the total of energy transfert coefficient to 100%.


In [9]:
constraint_energy = constraint.ConstraintNoTransportEnergyCoefficient(
    parameters_name=["D1N1_energy_coefficient", "D2N1_energy_coefficient"],
    min_energy_coef_value=0,
    max_energy_coef_value=1,
)

Finaly, create the Genetic Algorithm.


In [10]:
client = Client()
genetic_algo = GeneticAlgorithm(
    cost_function=cost_function,
    parameter_genetic_algorithm=genetic_algo_parameters,
    constraint=[constraint_energy],
    client=client,
)

2024-11-06 12:06:55,332 - tornado.application - ERROR - Exception in callback <bound method SystemMonitor.update of <SystemMonitor: cpu: 6 memory: 92 MB fds: 167>>
Traceback (most recent call last):
  File "/Users/ash/Documents/Workspaces/PhD/Seapopym-optimisation/.venv/lib/python3.12/site-packages/tornado/ioloop.py", line 937, in _run
    val = self.callback()
          ^^^^^^^^^^^^^^^
  File "/Users/ash/Documents/Workspaces/PhD/Seapopym-optimisation/.venv/lib/python3.12/site-packages/distributed/system_monitor.py", line 168, in update
    net_ioc = psutil.net_io_counters()
              ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ash/Documents/Workspaces/PhD/Seapopym-optimisation/.venv/lib/python3.12/site-packages/psutil/__init__.py", line 2139, in net_io_counters
    rawdict = _psplatform.net_io_counters()
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
OSError: [Errno 12] Cannot allocate memory


And watch the magic on the Dask dashboard :


In [11]:
genetic_algo.client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:57829,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:57840,Total threads: 2
Dashboard: http://127.0.0.1:57843/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:57832,


And execute the process.


In [12]:
viewer = genetic_algo.optimize()

2024-11-06 11:56:44,830 :: Seapodym ::  DEBUG ::
|	Direct computation for global_mask_from_nan.

2024-11-06 11:56:44,830 :: Seapodym ::  DEBUG ::
|	Direct computation for global_mask_from_nan.

2024-11-06 11:56:44,830 :: Seapodym ::  DEBUG ::
|	Direct computation for global_mask_from_nan.

2024-11-06 11:56:44,832 :: Seapodym ::  DEBUG ::
|	Direct computation for mask_by_fgroup.

2024-11-06 11:56:44,832 :: Seapodym ::  DEBUG ::
|	Direct computation for mask_by_fgroup.

2024-11-06 11:56:44,832 :: Seapodym ::  DEBUG ::
|	Direct computation for mask_by_fgroup.

2024-11-06 11:56:44,836 :: Seapodym ::  DEBUG ::
|	Direct computation for global_mask_from_nan.

2024-11-06 11:56:44,839 :: Seapodym ::  DEBUG ::
|	Direct computation for mask_by_fgroup.

2024-11-06 11:56:44,841 :: Seapodym ::  DEBUG ::
|	Direct computation for _wrapper_mesh_day_lengths.

2024-11-06 11:56:44,841 :: Seapodym ::  DEBUG ::
|	Direct computation for _wrapper_mesh_day_lengths.

2024-11-06 11:56:44,842 :: Seapodym ::  DEBU

Finaly here is the result :


In [13]:
viewer.logbook

,gen,nevals,avg,std,min,max,nvalide,ninvalide
0,0,20,3.196788e-07,9.099701e-08,1.685194e-07,5.010179e-07,12,8
1,1,9,2.790550e-07,8.183059e-08,1.685194e-07,5.010179e-07,18,2
2,2,13,2.242610e-07,8.411222e-08,1.387973e-07,5.010179e-07,18,2
3,3,11,2.023582e-07,8.734414e-08,1.318533e-07,5.010179e-07,20,0
4,4,12,2.678075e-07,3.564356e-07,1.318533e-07,1.484876e-06,20,0
5,5,16,1.532653e-07,4.366143e-08,1.318533e-07,3.390592e-07,20,0
6,6,10,1.518194e-07,5.833948e-08,1.318533e-07,4.049551e-07,20,0
7,7,10,1.349653e-07,3.283839e-09,1.318533e-07,1.408702e-07,20,0
8,8,12,1.411579e-07,3.488906e-08,1.318533e-07,2.929167e-07,20,0
9,9,14,1.370978e-07,2.059907e-08,1.318533e-07,2.267363e-07,20,0


In [14]:
viewer.box_plot(5)

In [15]:
viewer.parallel_coordinates(
    color_continuous_scale=[
        [0, "rgba(255,0,0,0)"],
        [0.8, "rgba(255,0,0,0.5)"],
        [1, "rgba(0, 255, 0, 1)"],
    ]
)